In [15]:
%run utils.ipynb
import spacy
from spacy import displacy
import torch
import numpy as np
import math
from thinc.api import set_gpu_allocator, require_gpu
set_gpu_allocator("pytorch")
require_gpu(0)

True

In [4]:
loader = DataLoader()
d = loader.GetData('final')

In [5]:
for i in range(len(d)):
    d['text'][i]= d['text'][i].replace("'","' ")

C:\Users\GIGA\AppData\Local\Temp\ipykernel_23336\2064060296.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['text'][i]= d['text'][i].replace("'","' ")


In [13]:
def GetAllComb(n,NumList):
    # Get all subsets of length n in the given list, and in the original order
    res = []
    if n == 1: # [0,1,2] --> [[0],[1],[2]]
        return [[i] for i in NumList]
    if n == len(NumList): # [0,1,2] --> [0,1,2]
        return [[i for i in NumList]]
    if n >= 2: # for each element before len(NumList)-n, insert that element into position 0 for all subsets of length n-1 of the remaining elements
        for start in range(len(NumList)-n+1): 
            # (3,[0,1,2,3,4])
            last = GetAllComb(n-1,NumList[start+1:])
            # 0 + (2,[1,2,3,4]), 1 + (2,[2,3,4]), 2 + (2,[3,4])
            for comb in last:
                comb.insert(0,NumList[start])
            res += last
        return res
def KernelFunction(tree1,tree2):
    # tree1 and tree2 are actually trees of token objects
    # get dep labels of all nodes, t1 and t2 are trees of dep labels
    t1 = (tree1[0].dep_,[i.dep_ for i in tree1[1] if i])
    t2 = (tree2[0].dep_,[i.dep_ for i in tree2[1] if i])
    # mu and lambda are decay factors, mu penalize tree height and lambda penalize tree length
    mu = 0.4
    lambda_ = 0.4
    k = 0 # final sum
    if t1[0] == t2[0]: # if labels are the same
        for tree_len in range(1,min(len(t1[1]),len(t2[1]))) :
            # length of all possible subtrees 
            prod = 1 # product
            ChildSeqs = GetAllComb(tree_len,[c for c in range(tree_len)])
            # list of indices of all possible child sequences of given length
            for j1 in ChildSeqs:
                for j2 in ChildSeqs:
                    # Get all pairs of sub sequences
                    for i in range(tree_len):
                        childt1 = tree1[1][j1[i]] # token object
                        childt2 = tree2[1][j2[i]]
                        prodt1 = (tree1[0],[j for j in childt1.children if j]) # build subtree 
                        prodt2 = (tree2[0],[j for j in childt2.children if j])
                        prod = prod * KernelFunction(prodt1,prodt2) 
                        # continue matching subtree
                    # finish matching indices j1 and j2, sum 
                    dt1 = j1[-1] - j1[0] + 1
                    dt2 = j2[-1] - j2[0] + 1    
                    k += (lambda_) ** (dt1 + dt2) * prod
        return mu * (lambda_**2 + k)
    else:
        return 0
def PartialTreeKernel(tree1,tree2):
    sim = 0
    for tokent1 in tree1:
        prodt1 = (tokent1,[i for i in tokent1.children if i])
        for tokent2 in tree2:
            # sum over all nodes
            prodt2 = (tokent2,[i for i in tokent2.children if i])
            if tokent1.text.isalpha() and (tokent1.dep_ == tokent2.dep_):
                sim += KernelFunction(prodt1,prodt2)
    return sim 
def normPTK(tree1,tree2):
    return PartialTreeKernel(tree1,tree2)/math.sqrt(PartialTreeKernel(tree1,tree1)*PartialTreeKernel(tree2,tree2))

In [7]:
NLP = spacy.load('en_core_web_trf')

In [33]:
sim_list = []
for num in range(len(d)):
    doc = NLP(d['text'][num])
    sents = [j for j in doc.sents if len(str(j).split()) > 5 ]   
    sim_matrix= np.eye(len(sents),len(sents))
    for i in range(len(sents)):
        for j in range(i+1,len(sents)):
            sim_matrix[i,j] = normPTK(sents[i],sents[j])
    if num % 50 == 0:
        print(num)
    sim_list.append(sim_matrix)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250


Token indices sequence length is longer than the specified maximum sequence length for this model (662 > 512). Running this sequence through the model will result in indexing errors


1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800


In [34]:
np.savez('final_winter_sent.npz',*sim_list)

In [38]:
f = np.load('features/final_winter_sent.npz')